SreeDananjay S(21BAI1807)
Speech and Language processing Lab assignment 6

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

2024-09-09 20:54:20.733847: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-09 20:54:21.231457: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 20:54:21.231525: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 20:54:21.306940: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-09 20:54:21.529335: I tensorflow/core/platform/cpu_feature_guar

In [3]:
df = pd.read_csv('SMSSpamCollection', sep='\t', names=['label', 'message'])

In [4]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [5]:
X = df['message'].values
y = df['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
vocab_size = 10000  # You can adjust this based on your dataset size
max_len = 100       # Maximum length of sequences after padding
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [7]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure they are of equal length
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post', truncating='post')

In [9]:
embedding_dim = 128

model = Sequential()

# Embedding layer to convert word indices into dense vectors
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))

# Add a simple RNN layer
model.add(SimpleRNN(128, return_sequences=False))

# Add a dropout layer to prevent overfitting
model.add(Dropout(0.5))

# Add the output layer with a sigmoid activation function for binary classification (spam/ham)
model.add(Dense(1, activation='sigmoid'))

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
batch_size = 64
epochs = 5
history = model.fit(X_train_padded, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test_padded, y_test))

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test_padded, y_test, verbose=0)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Making predictions on the test set
predictions = model.predict(X_test_padded)

# Check first few predictions
for i in range(5):
    print(f"Message: {X_test[i]}")
    print(f"Actual: {y_test[i]}, Predicted: {1 if predictions[i] > 0.5 else 0}\n")

Epoch 1/5
70/70 [==============================] - 4s 44ms/step - loss: 0.4417 - accuracy: 0.8512 - val_loss: 0.4053 - val_accuracy: 0.8655
Epoch 2/5
70/70 [==============================] - 3s 39ms/step - loss: 0.4147 - accuracy: 0.8658 - val_loss: 0.3937 - val_accuracy: 0.8664
Epoch 3/5
70/70 [==============================] - 3s 40ms/step - loss: 0.3479 - accuracy: 0.8665 - val_loss: 0.4539 - val_accuracy: 0.8475
Epoch 4/5
70/70 [==============================] - 3s 38ms/step - loss: 0.2060 - accuracy: 0.9248 - val_loss: 0.2447 - val_accuracy: 0.9238
Epoch 5/5
70/70 [==============================] - 3s 36ms/step - loss: 0.3940 - accuracy: 0.8600 - val_loss: 0.3264 - val_accuracy: 0.8664
Test Accuracy: 86.64%
35/35 [==============================] - 0s 7ms/step
Message: Squeeeeeze!! This is christmas hug.. If u lik my frndshp den hug me back.. If u get 3 u r cute:) 6 u r luvd:* 9 u r so lucky;) None? People hate u:
Actual: 0, Predicted: 0

Message: And also I've sorta blown him off 